# Simulating interviews with prior survey respondents
This notebook provides sample [EDSL](https://docs.expectedparrot.com/) code for creating AI agents representing respondents of your existing survey data, and automating simulated follow-on interviews with them.

EDSL is an open-source libary for simulating surveys experiments with AI agents and language models. Please see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started.

## Importing prior survey data
We start by importing a dataset of responses from an existing survey. For purposes of demonstration, we use a set of mock responses to a survey about a popular product management newsletter that consisted of the following questions:

1. *Multiple choice:* **How often do you read the newsletter?** Options: Daily, Weekly, Monthly, Rarely, Never

2. *Free text:* **What topics would you like to see covered in future issues?**

3. *Linear scale:* **On a scale of 1 to 10, how would you rate the overall quality of the newsletter?** Options: 1-10

4. *Multiple choice:* **Which section of the newsletter do you find most valuable?** Options: Product updates, Industry news, Case studies, Tips and strategy

5. *Free text:* **What improvements would you suggest for the newsletter?**

Reading in the CSV:

In [1]:
import pandas as pd

df = pd.read_csv("product_manager_responses.csv")

Checking the questions:

In [2]:
df[["question_type", "question_text", "question_options"]].drop_duplicates()

,question_type,question_text,question_options
0,Multiple Choice,How often do you read the newsletter?,"Daily, Weekly, Monthly, Rarely, Never"
5,Free Text,What topics would you like to see covered in f...,NaN
10,Linear Scale,"On a scale of 1 to 10, how would you rate the ...",1-10
15,Multiple Choice,Which section of the newsletter do you find mo...,"Product Updates, Industry News, Case Studies, ..."
20,Free Text,What improvements would you suggest for the ne...,NaN


Inspecting a sample of the responses:

In [3]:
df[["question_text", "response"]].sample(5)

,question_text,response
21,What improvements would you suggest for the ne...,The visual design of the newsletter could be i...
11,"On a scale of 1 to 10, how would you rate the ...",7
17,Which section of the newsletter do you find mo...,Case Studies
18,Which section of the newsletter do you find mo...,Tips and Strategies
15,Which section of the newsletter do you find mo...,Tips and Strategies


## Create agents for the interview subjects
Here we create an `Agent` for each `respondent_id` and give it information about its responses to the original survey. We do this by reformatting each set of responses as a dictionary of `traits` that we pass to an agent:

In [4]:
# Change the set of all responses into a dictionary
all_responses = df.to_dict(orient="records") 

# Initialize a dictionary to store responses by respondent_id
respondents_dict = {}

# Iterate over each row in the responses
for row in all_responses:
    respondent_id = row['respondent_id']
    question_id = row['question_id']
    question_text = row['question_text']
    question_options = row['question_options']
    response = row['response']
    
    # Format each response as background information for the relevant respondent
    formatted_string = (
        f"You were asked: '{question_text}' "
        f"The answer options were: '{question_options}'. "
        f"You responded: '{response}'"
    )
    
    # Add the information to the respondent's dictionary
    if respondent_id not in respondents_dict:
        respondents_dict[respondent_id] = {}
    
    respondents_dict[respondent_id][question_id] = formatted_string

# Print the new dictionary 
print(respondents_dict)

{1: {1: "You were asked: 'How often do you read the newsletter?' The answer options were: 'Daily, Weekly, Monthly, Rarely, Never'. You responded: 'Weekly'", 2: "You were asked: 'What topics would you like to see covered in future issues?' The answer options were: 'nan'. You responded: 'I would love to see more in-depth analyses on emerging trends in product management. It would be beneficial to include case studies from various industries, as it helps to see how different strategies are implemented in real-world scenarios. Additionally, insights from seasoned product managers on how they tackle common challenges would be greatly appreciated.'", 3: "You were asked: 'On a scale of 1 to 10, how would you rate the overall quality of the newsletter?' The answer options were: '1-10'. You responded: '8'", 4: "You were asked: 'Which section of the newsletter do you find most valuable?' The answer options were: 'Product Updates, Industry News, Case Studies, Tips and Strategies'. You responded: 

Next we pass the dictionaries to `Agent` objects, together with information and instructions about the follow-on interviews that we want to conduct:

In [5]:
from edsl import Agent

import textwrap
from rich import print

# Interview topic
interview_topic = "Product Management Newsletter Follow-up Interview"

# Persona for the interview subjects
interview_subject_persona = textwrap.dedent(f"""\
You are a professional with a keen interest in product management.
""")

# Instructions for the interview subject agents
interview_subject_instructions = textwrap.dedent(f"""\
You recently completed a reader survey about the newsletter of a 
product management expert who also produces a popular blog and podcast. 
Now they are asking you some follow-on questions.
""")

# Persona for the interviewer agent
interviewer_persona = textwrap.dedent(f"""\
You are an well-known expert on product management who produces 
a popular newsletter, blog and podcast on product management.  
""")

# Instructions for the interviewer agent
interviewer_instructions = textwrap.dedent(f"""\
You recently conducted a reader survey about your newsletter. 
Now you are asking respondents some follow-on questions.
""")

# Total number of questions to ask in the interview
total_questions = 5 

In [6]:
# Initialize a list to store the agents
interview_subjects = []

# Iterate over the respondents' data
for respondent_id, questions in respondents_dict.items():
    
    # Initialize the traits for each agent
    traits = {}
    
    # Iterate over the questions
    for question_id, formatted_string in questions.items():
        traits[f"question_id_{question_id}"] = formatted_string
    
    # Create the agent and add it to the agents list
    agent = Agent(name = f"Respondent {respondent_id}", 
                  traits = traits, 
                  instruction = interview_subject_instructions)
    interview_subjects.append(agent)

interview_subjects

[Agent(name = 'Respondent 1', traits = {'question_id_1': "You were asked: 'How often do you read the newsletter?' The answer options were: 'Daily, Weekly, Monthly, Rarely, Never'. You responded: 'Weekly'", 'question_id_2': "You were asked: 'What topics would you like to see covered in future issues?' The answer options were: 'nan'. You responded: 'I would love to see more in-depth analyses on emerging trends in product management. It would be beneficial to include case studies from various industries, as it helps to see how different strategies are implemented in real-world scenarios. Additionally, insights from seasoned product managers on how they tackle common challenges would be greatly appreciated.'", 'question_id_3': "You were asked: 'On a scale of 1 to 10, how would you rate the overall quality of the newsletter?' The answer options were: '1-10'. You responded: '8'", 'question_id_4': "You were asked: 'Which section of the newsletter do you find most valuable?' The answer options

## Create an interviewer agent

In [7]:
# Create an agent for the interviewer 
interviewer = Agent(traits = {"persona":interviewer_persona},
                    instruction = interviewer_instructions)

## Methods for automating the interviews

In [8]:
from edsl.questions import QuestionFreeText
from edsl import Scenario, Model, Survey

# Selecting a language model to use
model = Model('gpt-4')

def get_next_question(subject, researcher, dialog_so_far):
    scenario = Scenario({'subject': str(subject.traits), 'dialog_so_far': dialog_so_far})
    meta_q = QuestionFreeText(
        question_name="next_question",
        question_text="""
        This is the background information for the interview subject: {{ subject }}
        This is your current dialog with the interview subject: {{ dialog_so_far }}
        What question you would ask the interview subject next?
        """
    )
    question_text = meta_q.by(model).by(researcher).by(scenario).run().select("next_question").first()
    return question_text

def get_response_to_question(question_text, subject, dialog_so_far):
    q_to_subject = QuestionFreeText(
        question_name="question",
        question_text=f"""
        This is your current dialog with the interview subject: {dialog_so_far}.
        You are now being asked:""" + question_text
    )
    response = q_to_subject.by(model).by(subject).run().select("question").first()
    return response

def ask_question(subject, researcher, dialog_so_far):
    question_text = get_next_question(subject, researcher, dialog_so_far)
    response = get_response_to_question(question_text, subject, dialog_so_far)

    print(" \nQuestion: \n\n" + question_text + "\n\nResponse: \n\n" + response)
    
    return {"question": question_text, "response": response}

def dialog_to_string(d):
    return "\n".join([f"Question: {d['question']}\nResponse: {d['response']}" for d in d])

def clean_dict(d):
    """Convert dictionary to string and remove braces."""
    return str(d).replace('{', '').replace('}', '')

def summarize_interview(subject, interview_topic, dialog_so_far, researcher):
    interview_subject_name = subject["name"]
    interview_subject_traits = subject["traits"]
    summary_q = QuestionFreeText(
        question_name = "summary",
        question_text = (
        f"You have just conducted the following interview of {interview_subject_name} "
        f"who has these traits: {clean_dict(interview_subject_traits)} "
        f"The topic of the interview was {interview_topic}. "
        f"Please draft a summary of the interview: {clean_dict(dialog_so_far)}")
    )
    themes_q = QuestionFreeText(
        question_name = "themes",
        question_text = "List the major themes of the interview."
    )
    survey = Survey([summary_q, themes_q]).set_full_memory_mode()
    results = survey.by(model).by(researcher).run()
    summary = results.select("summary").first()
    themes = results.select("themes").first()
    print("\n\nSummary:\n\n" + summary + "\n\nThemes:\n\n" + themes)

def conduct_interview(subject, researcher, interview_topic):

    print("\n\nInterview subject: " + subject["name"] + "\n\nInterview topic: " + interview_topic)
    
    dialog_so_far = []  
    
    for i in range(total_questions):
        result = ask_question(subject, researcher, dialog_to_string(dialog_so_far))
        dialog_so_far.append(result)
    
    summarize_interview(subject, interview_topic, dialog_so_far, researcher)

In [9]:
for interview_subject in interview_subjects:
    conduct_interview(interview_subject, interviewer, interview_topic)

Interview subject: Respondent 1

Interview topic: Product Management Newsletter Follow-up Interview

Question: 

Thank you for your valuable feedback. Based on your responses, we understand you find the 'Tips and Strategies' 
section most valuable and you'd like to see more in-depth analyses and case studies. Could you please elaborate on 
the specific topics under 'Tips and Strategies' that you find most helpful or would like to see more of? Also, 
regarding the interactive content, what format would you prefer? Do you like live webinars, recorded sessions, or 
written Q&A sessions?

Response: 

Under the 'Tips and Strategies' section, I find the practical advice on how to manage product development cycles, 
stakeholder communication, and user experience design very helpful. I would love to see more content on how to 
effectively use data in decision making, strategies for managing remote product teams, and best practices for 
product marketing. As for the interactive content, I would prefer a mix of live webinars and written Q&A sessions. 
Live webinars provide an opportunity to interact in real-time and get immediate answers, while written Q&A sessions
allow for more thoughtful responses and can be referred back to later.

Question: 

Thank you for your detailed response. I'm glad to hear that you find our content on product development cycles, 
stakeholder communication, and user experience design useful. Your suggestions on topics such as data-driven 
decision making, remote team management, and product marketing are noted. As for the interactive content, we'll 
consider incorporating a mix of live webinars and written Q&A sessions. To help us further improve, could you 
please share what you think makes a webinar engaging and effective? Also, what type of questions would you like to 
see answered in the Q&A sessions?

Response: 

For a webinar to be engaging and effective, it should be interactive with opportunities for attendees to ask 
questions and participate in discussions. The use of visuals and real-life examples can also make the content more 
relatable and easier to understand. As for the Q&A sessions, I would like to see answers to questions about common 
challenges faced by product managers, such as how to prioritize features, manage scope creep, and handle conflicts 
within the team. It would also be interesting to hear about the experiences of seasoned product managers and the 
lessons they've learned throughout their careers.

Question: 

Thank you for your thoughtful feedback. Interactive and relatable content is indeed crucial in webinars. Your 
suggestions for Q&A topics are very insightful and will definitely help us in planning future sessions. One last 
question, would you be interested in contributing to our newsletter, perhaps sharing a case study from your own 
experience or writing a guest post on a topic of your choice? This could provide a fresh perspective to our readers
and foster a sense of community among product managers.

Response: 

I appreciate the invitation and I would be interested in contributing to the newsletter. I believe I could share a 
case study from a recent product launch in my company that had some unique challenges. This might provide some 
practical insights for other product managers. I will need some time to prepare the material, but I look forward to
the opportunity to engage with the community in this way.

Question: 

That sounds fantastic! We'd love to learn from your experiences and I'm sure our readers would too. Please take 
your time in preparing the material. Could you give us an idea of the timeline you'd need? Also, would you be open 
to having a Q&A session or a discussion with our readers following the publication of your case study? This could 
provide an opportunity for our readers to delve deeper into the topic and ask any questions they might have.

Response: 

I would need about a month to prepare the material for the case study. As for the Q&A session or discussion 
following the publication, I would be open to that idea. I believe it would be a great way to engage with the 
readers and answer any questions they might have about the case study. I look forward to contributing to your 
newsletter and interacting with your audience.

...

Question: 

Thank you for sharing those insights. It's really helpful to understand your interest in predictive analytics, 
machine learning, and data visualization as they relate to product management. We'll certainly consider 
incorporating tutorials and insights on these topics. As for the visual design, we'll take a look at 'Inside 
Design' and 'Morning Brew' for inspiration. Could you please share what you like about the visual design of these 
newsletters in more detail? Also, are there any specific elements or features you would like to see in our 
newsletter's design?

Response: 

What I appreciate about the visual design of 'Inside Design' and 'Morning Brew' is their clean and uncluttered 
layout. They make good use of white space, which makes the content easy to read and navigate. Their typography is 
also well-chosen, with clear headings and subheadings that guide the reader through the content. The use of images 
and graphics in these newsletters is balanced and purposeful. They use visuals not just as decorations, but as 
integral parts of the content that help to illustrate points and break up the text. In terms of specific elements 
or features for your newsletter's design, I would suggest considering a consistent color scheme that aligns with 
your brand. Also, it would be beneficial to have a clear structure with defined sections for different types of 
content, like industry news, case studies, and tips. This would make it easier for readers to find the information 
they're interested in. Lastly, I believe that incorporating more data visualizations and infographics could make 
your content more engaging and easier to digest, especially when discussing complex topics like advanced analytics.

Question: 

Thank you for your detailed feedback on the visual design. Your suggestions are very helpful. In terms of content, 
you mentioned a desire for more advanced analytics techniques. Could you share any specific challenges or questions
you have encountered in your work that you would like addressed in these tutorials? This way, we can ensure our 
content is as relevant and practical as possible for our readers. Also, are there any experts or professionals in 
the field of advanced analytics that you would like to hear from in our interviews?

Response: 

In my work, one challenge I often encounter is determining the right data analytics techniques to use in different 
scenarios. For example, when is it more appropriate to use predictive analytics versus machine learning? How do you
choose the right data visualization to accurately represent your data and findings? It would be helpful if the 
tutorials could provide guidance on these types of questions. Another challenge is keeping up with the latest tools
and software for data analysis and visualization, as the landscape is constantly evolving. Tutorials that provide 
updates on the latest tools and how to use them would be very useful. As for experts in the field of advanced 
analytics, I would be interested in hearing from professionals who have successfully applied these techniques in 
product management. It would be great to learn from their experiences and get their insights on best practices. 
Additionally, it would be interesting to hear from experts who are pushing the boundaries of what's possible with 
advanced analytics, like those working on AI and machine learning research.

Question: 

Thank you for sharing the challenges you face in your work and the types of experts you'd like to hear from. We 
will certainly aim to address these topics in our future content. To ensure that we deliver the most relevant and 
practical information, could you please share a bit more about your current role, the industry you're in, and the 
size and type of the projects you typically work on? This will help us tailor our content to your specific needs. 
Additionally, are there any other topics or sections you'd like to see in the newsletter that we haven't discussed 
yet?

Response: 

I currently work as a Senior Product Manager in the tech industry, specifically in a company that develops SaaS 
products. The projects I typically work on vary in size, but they often involve cross-functional teams of 10-20 
people and span several months from conception to launch. In terms of other topics or sections for the newsletter, 
I would be interested in seeing more content on leadership and team management in a product management context. 
This could include tips on how to effectively lead a product team, manage conflicts, and foster a collaborative and
innovative team culture. I also think a section on 'Product Management Tools and Resources' would be useful, where 
you could share recommendations on books, courses, tools, and other resources for product managers.

Question: 

Thank you for sharing more about your role and your interest in leadership and team management topics, as well as 
resources for product managers. We will certainly consider adding these to our content. Given your role and the 
size of the projects you work on, are there any specific challenges related to leadership or team management that 
you frequently encounter? Also, could you share any books, courses, tools, or other resources that you have found 
particularly useful in your work as a Senior Product Manager? This will help us understand the kind of 
recommendations that might be most valuable to our readers.

Response: 

In my role, some of the leadership and team management challenges I frequently encounter include aligning the team 
around a shared vision, managing conflicts among team members, and maintaining open and effective communication 
across different functions. It would be helpful to see content that offers strategies and best practices for 
addressing these challenges. As for resources, 'Inspired: How to Create Tech Products Customers Love' by Marty 
Cagan and 'The Lean Product Playbook' by Dan Olsen have been invaluable in my work. In terms of tools, I frequently
use Jira for project management, Slack for team communication, and Tableau for data visualization. A course that I 
found particularly useful was the 'Product Strategy' course on Coursera by IE Business School. It provided a great 
framework for developing and executing a product strategy. I believe recommendations like these would be very 
valuable to your readers, especially those who are new to the field or looking to deepen their knowledge in 
specific areas of product management.

Summary:

Respondent 5, a monthly reader of our product management newsletter, rated its overall quality as 7 out of 10 and 
finds the 'Industry News' section most valuable. They suggested a series on advanced analytics techniques, 
including predictive analytics, machine learning, and data visualization, with tutorials on tools like Python, R, 
and Tableau. They also expressed interest in hearing from experts who have successfully applied these techniques in
product management. Respondent 5 would like improvements in the newsletter's visual design, inspired by 'Inside 
Design' and 'Morning Brew', with a clean, uncluttered layout, good use of white space, clear typography, balanced 
use of images and graphics, and a consistent color scheme. They also suggested a clear structure with defined 
sections and more data visualizations and infographics. Further suggestions included content on leadership and team
management in product management and a section on 'Product Management Tools and Resources'. They work as a Senior 
Product Manager in the tech industry and often face challenges in choosing right analytics techniques, keeping up 
with the latest tools, aligning the team around a shared vision, managing conflicts, and maintaining effective 
communication.

Themes:

Respondent 5, a monthly reader of our product management newsletter, rated its overall quality as 7 out of 10 and 
finds the 'Industry News' section most valuable. They suggested a series on advanced analytics techniques, 
including predictive analytics, machine learning, and data visualization, with tutorials on tools like Python, R, 
and Tableau. They also expressed interest in hearing from experts who have successfully applied these techniques in
product management. Respondent 5 would like improvements in the newsletter's visual design, inspired by 'Inside 
Design' and 'Morning Brew', with a clean, uncluttered layout, good use of white space, clear typography, balanced 
use of images and graphics, and a consistent color scheme. They also suggested a clear structure with defined 
sections and more data visualizations and infographics. Further suggestions included content on leadership and team
management in product management and a section on 'Product Management Tools and Resources'. They work as a Senior 
Product Manager in the tech industry and often face challenges in choosing right analytics techniques, keeping up 
with the latest tools, aligning the team around a shared vision, managing conflicts, and maintaining effective 
communication.